### Test Notebook for running Cortex AI with local code

To run this notebook, 

1. Make sure to have Python 3.12.4 installed. That is the latest version with a wheel as seen in this post (https://stackoverflow.com/questions/79135647/pip-install-snowflake-connector-python-fails-building-wheels). Using later versions will demand an installation of Visual Studio C++ 14.0, which i didn't test.

2. create an environment and install the requirements.txt file

3. Create Snowflake credentials in windows credential manager (Follow this post: https://medium.com/@aarhar/password-management-in-python-keyring-and-credential-manager-29fa4ccc919e)

4. Run a query to test you connection (Currently it's not working, but the "basics" are there.)

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keyring
import os 
import snowflake.connector as sf_connector # ( https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-connect)
from snowflake.connector.pandas_tools import write_pandas # (https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-api#write_pandas)
import fitz 
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
from collections import defaultdict
from PIL import Image
from io import BytesIO
import layoutparser as lp
import cv2
import numpy as np
from tqdm import tqdm

## TLDR: This notebook

The idea of this notebook is to create NLP and image oriented functions focused on the application of extracting and strcturing information from a pdf.

It currently contains the following type of functions:
- A setup/config block of code intended to connect with snowflake databases.
- A suggestion of the database structure to store the information extracted from the pdf.
- `extract_manual_metadata()` : Extracts metadata from a PDF file using PyMuPDF.
- `explore_pdf_fonts_llm_ready()` : Creates a string block of text summarizing the fonts and font sizes used in a PDF file, along with examples. The intended use of this function is to potentially feed this string to a RAG model, to help it understand the structure of the PDF, and perhaps enable the model to label sections of the text, such as "title", "subtitle", "paragraph", "warnings", etc.
- `extract_text_chunks()` : Extracts all the text data from a PDF file using PyMuPDF, and splits it in a dataframe in string chunks. Could potentially be replaced by CORTEX from Snowflake for more accurate results.
- `extract_images_from_pdf()` : Extracts all the images from a PDF using a sequence of functions. 
    1. Each PDF are rendered as a list of images using `render_pdf_to_images()`. 
    2. Then each page is processed using `detect_image_regions()`, which runs using classic computer vision techniques:
        2.1 `cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)` converts the image to grayscale.
        2.2 `cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)` applies a binary threshold to the grayscale image, meaning that colors above 240 are set to 0 (black) and all values below 240 are set to 255 (white). This creates "black boxes" around the images, and leaves the text areas white.
        2.3 `cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)` finds the contours of the black boxes, and returns a list of coordinates for each box.
    3. Using the list of coordinates for the "black regions", which are presumed to be images, the function `crop_regions_from_image()` crops the original page leaving only the image in the region of interest. These images are then saved in a folder corresponding to the file name, and the function returns a list of metadata to the images.


### Notes and considerations: 
- The image extraction process intentionally avoids using modern object segmentation techniques like YOLO or Mask R-CNN, as i found out that these methods have a lot of dependencies which i couldn't make compatible with python 3.12.4. It might be possible, but i believe that the current method of calculating contours is possibly more effective, and orders of magnitude faster than using a deep learning model. This has only been tested on 1 PDF, hence more testing is needed to rely on the methods.


### Import issues 

- PyMuPDF and fitz has proved to be slightly frustrating. The best solution i found so far was after installing requirements.txt is to:

```pythonpip uninstall fitz \n pip install``` 

```--upgrade --force-reinstall pymupdf```

In [32]:
account_identifier = keyring.get_password('NC_Snowflake_Trial_Account_Name', 'account_identifier')
user_name = "EMILHALDAN5468402"
password = keyring.get_password('NC_Snowflake_Trial_User_Password', user_name)

print("Account Identifier: ", account_identifier)
print("User Name: ", user_name)
# print("Password: ", password)


Account Identifier:  EDBHJWL-MFB05236
User Name:  EMILHALDAN5468402


In [33]:

connection_parameters = {
    "account_identifier": account_identifier,
    "user": user_name,
    "password": password,
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "CYBERSYN",
    "schema": "PUBLIC"
}

# Connect to Snowflake
conn = sf_connector.connect(
    user=connection_parameters['user'],
    password=connection_parameters['password'],
    account=connection_parameters['account_identifier'],
    warehouse=connection_parameters['warehouse'],
    database=connection_parameters['database'],
    schema=connection_parameters['schema'],
    role=connection_parameters['role']
)

# # Test the connection
cursor = conn.cursor()
cursor.execute("""
    SELECT * 
    FROM company_metadata;
""")

df = cursor.fetch_pandas_all()
df.head(3)


,CYBERSYN_COMPANY_ID,COMPANY_NAME,PERMID_SECURITY_ID,PRIMARY_TICKER,SECURITY_NAME,ASSET_CLASS,PRIMARY_EXCHANGE_CODE,PRIMARY_EXCHANGE_NAME,SECURITY_STATUS,GLOBAL_TICKERS,EXCHANGE_CODE,PERMID_QUOTE_ID
0,07e85915f5b12be5f0f3fe67276ffc08,"CELSIUS HOLDINGS, INC.",8589948642,CELH,CELSIUS HOLDINGS ORD SHS,Ordinary Shares,NAS,NASDAQ CAPITAL MARKET,Active,"[\n ""CELH""\n]","[\n ""NAS""\n]","[\n ""25727435733""\n]"
1,0ac135dfdf2b133215109ccc15754a62,CHURCH & DWIGHT CO INC /DE/,8590943794,CHD,CHURCH AND DWIGHT ORD SHS,Ordinary Shares,NYS,NEW YORK STOCK EXCHANGE,Active,"[\n ""0R13"",\n ""0R13l"",\n ""CHD"",\n ""CHD*"",\...","[\n ""BCO"",\n ""LSE"",\n ""MCX"",\n ""MXQ"",\n ""...","[\n ""21550585218"",\n ""21634298419"",\n ""2167..."
2,1443e2413b8a7cb8126ec02fb9096da6,GENERAL MILLS INC,8590933269,GIS,GENERAL MILLS ORD SHS,Ordinary Shares,NYS,NEW YORK STOCK EXCHANGE,Active,"[\n ""0R1X"",\n ""0R1Xl"",\n ""GIS"",\n ""GIS*"",\...","[\n ""BCO"",\n ""BIV"",\n ""LSE"",\n ""MCX"",\n ""...","[\n ""21550585154"",\n ""21634298003"",\n ""2165..."


## The intended database structure is as follows:

- **manuals** (Stores metadata about each manual)  
  - `manual_id` (Unique ID for each manual)  
  - `doc_name` ()
  - `title` (Title of the manual)  
  - `version` (Version or revision number)  
  - `language` (Language code, e.g., 'en', 'de')  
  - `source_path` (Original PDF file path or S3 URL)   

- **sections** (Defines logical sections and subsections within each manual)  
  - `section_id` (Unique ID for the section)  
  - `manual_id` (Foreign key referencing `manuals`)  
  - `title` (Title or heading of the section)  
  - `order_num` (Numerical order of the section in the manual)  
  - `parent_section_id` (Optional FK for nested subsections)  

- **chunks small** (512 characters, 64 overlap)
  - `chunk_id` (Unique ID for the chunk)  
  - `section_id` (Foreign key referencing `sections`)  
  - `chunk_text` (The text content of the chunk)  
  - `chunk_order` (Order of the chunk within the section)  
  - `embedding` (Vector for semantic search or embeddings)  

- **chunks large** (1024 characters)
  - `chunk_id` (Unique ID for the chunk)  
  - `section_id` (Foreign key referencing `sections`)  
  - `chunk_text` (The text content of the chunk)  
  - `chunk_order` (Order of the chunk within the section)  
  - `embedding` (Vector for semantic search or embeddings)  

- **images** (Stores references to images extracted from the manual)  
  - `image_id` (Unique ID for the image)  
  - `manual_id` (Foreign key referencing `manuals`)  
  - `section_id` (Foreign key referencing `sections`)  
  - `image_path` (S3 or web-accessible path to the image)  
  - `caption` (Optional caption or alt text)  
  - `order_num` (Display order within the section)  


In [34]:
sections_df = pd.DataFrame(columns=['Section_ID', 'Manual_ID', 'Doc_Name', 'Order_Num', 'Parent_Section_ID'])
images_df = pd.DataFrame(columns=['Image_ID', 'Section_ID', 'Manual_ID', 'Order_Num', 'Image_Description' ,'Image_Path'])

### This section involves creating the database structure in Snowflake.

In [79]:
# # Test the connection
cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS WASHING_MACHINE_MANUALS;")
cursor.execute("CREATE SCHEMA IF NOT EXISTS PUBLIC;") 

cursor.execute("""
    CREATE OR REPLACE TABLE manuals (
    manual_id INT IDENTITY(1,1) PRIMARY KEY,
    doc_name VARCHAR(255) NOT NULL,
    title VARCHAR(255),
    version VARCHAR(255),
    language VARCHAR(50),
    source_path VARCHAR(1000) NOT NULL,
    created_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
    updated_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
);
""")

cursor.execute("""
    CREATE OR REPLACE TABLE sections (
    section_id INT IDENTITY(1,1) PRIMARY KEY,
    manual_id INT NOT NULL,
    title VARCHAR(255),
    order_num INT,
    parent_section_id INT,
    FOREIGN KEY (manual_id) REFERENCES manuals(manual_id),
    FOREIGN KEY (parent_section_id) REFERENCES sections(section_id)
);
""")

cursor.execute("""
    CREATE OR REPLACE TABLE chunks_large (
    chunk_id INT IDENTITY(1,1) PRIMARY KEY,
    section_id INT NOT NULL,
    manual_id INT NOT NULL,
    chunk_text VARCHAR(16777216),
    chunk_order INT,
    FOREIGN KEY (section_id) REFERENCES sections(section_id),
    FOREIGN KEY (manual_id) REFERENCES manuals(manual_id)
);
""")

cursor.execute("""
    CREATE OR REPLACE TABLE chunks_small (
    chunk_id INT IDENTITY(1,1) PRIMARY KEY,
    section_id INT NOT NULL,
    manual_id INT NOT NULL,
    chunk_text VARCHAR(16777216),
    chunk_order INT,
    FOREIGN KEY (section_id) REFERENCES sections(section_id),
    FOREIGN KEY (manual_id) REFERENCES manuals(manual_id)
);
""")

cursor.execute("""
    CREATE OR REPLACE TABLE images (
    image_id INT IDENTITY(1,1) PRIMARY KEY,
    manual_id INT NOT NULL,
    section_id INT NOT NULL,
    image_path VARCHAR(1000) NOT NULL,
    caption VARCHAR(1000),
    order_num INT,
    FOREIGN KEY (manual_id) REFERENCES manuals(manual_id),
    FOREIGN KEY (section_id) REFERENCES sections(section_id)
);
""")


In [6]:
pdf_files_path = ".\\Washer_Manuals"

for filename in os.listdir(pdf_files_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(pdf_files_path, filename)
        print(file_path)
        # do something with the file
    else:
        continue

.\Washer_Manuals\k714wm14 service manual.pdf
.\Washer_Manuals\mmo_87050793_1630397705_64_10689.pdf
.\Washer_Manuals\technical-manual-w11663204-revb.pdf
.\Washer_Manuals\WAK20160IN.pdf
.\Washer_Manuals\WAN28258GB.pdf
.\Washer_Manuals\WAN28282GC.pdf
.\Washer_Manuals\Washing machine Top-loader C series.pdf
.\Washer_Manuals\WAT24168IN.pdf
.\Washer_Manuals\WAV28KH3GB.pdf
.\Washer_Manuals\WFL2050.pdf
.\Washer_Manuals\WGA1340SIN.pdf
.\Washer_Manuals\WGA1420SIN.pdf
.\Washer_Manuals\WGE03408GB.pdf
.\Washer_Manuals\WGG254Z0GB.pdf


In [80]:
## Extracting metadata from the PDF files

def extract_manual_metadata(file_path):
    doc = fitz.open(file_path)
    metadata = doc.metadata
    file_name = file_path.split("\\")[-1]

    return {
        "DOC_NAME" : file_name,
        "TITLE": metadata.get("title") or file_path.split("/")[-1],
        "VERSION": metadata.get("modDate") or "v1",
        "LANGUAGE": "en",  # Default, (can be imrpoved later)
        "SOURCE_PATH": file_path,
    }

manuals_df = pd.DataFrame(columns=['DOC_NAME', 'TITLE', 'VERSION', 'LANGUAGE', 'SOURCE_PATH'])
extract_manual_metadata(file_path)


# fill the pandas dataframe with the metadata, and upload to snowflake
for filename in os.listdir(pdf_files_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(pdf_files_path, filename)
        metadata = extract_manual_metadata(file_path)
        manuals_df = pd.concat([manuals_df, pd.DataFrame([metadata])], ignore_index=True)

manuals_df.head()

write_pandas(df =manuals_df, conn= conn, 
                database = "WASHING_MACHINE_MANUALS",
                schema="PUBLIC",
                table_name= "MANUALS",  
                auto_create_table=False, 
                overwrite=False)


(True,
 1,
 14,
 [('tbzdytttup/file0.txt', 'LOADED', 14, 14, 1, 0, None, None, None, None)])

In [81]:
manuals_df.head()

,DOC_NAME,TITLE,VERSION,LANGUAGE,SOURCE_PATH
0,k714wm14 service manual.pdf,"维修手册_SERVICE MANUAL-SICILY GLORY(6.0,7.0KG)",D:20140618154138+08'00',en,.\Washer_Manuals\k714wm14 service manual.pdf
1,mmo_87050793_1630397705_64_10689.pdf,.\Washer_Manuals\mmo_87050793_1630397705_64_10...,D:20210812175917+09'00',en,.\Washer_Manuals\mmo_87050793_1630397705_64_10...
2,technical-manual-w11663204-revb.pdf,.\Washer_Manuals\technical-manual-w11663204-re...,D:20230606105253-04'00',en,.\Washer_Manuals\technical-manual-w11663204-re...
3,WAK20160IN.pdf,untitled,D:20140808160227+02'00',en,.\Washer_Manuals\WAK20160IN.pdf
4,WAN28258GB.pdf,User manual and installation instructions WAN2...,D:20250224094441+01'00,en,.\Washer_Manuals\WAN28258GB.pdf


In [13]:
## Exploring the PDF files and their structure

def explore_pdf_fonts_llm_ready(file_path : str, sort_by="size", examples_included:int = 8, verbose:int=0 ) -> str:
    """
    Args:
        file_path (str): A string containing the local path to the PDF file.
        sort_by (str, optional): Defaults to "size".
        verbose (int, optional): Option are (0,1,2) and increases amount of information printed, typically used for confirming documents read. Defaults to 0.

    Returns:
        str: _description_
    """

    doc = fitz.open(file_path)
    font_data = defaultdict(list)

    for p_idx,page in enumerate(doc):
        if verbose > 1:
            # used to show the pages processed to compare with the actual document
            print(f"Processing page {p_idx + 1}/{len(doc)}...")
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            for line in b.get("lines", []):
                for span in line.get("spans", []):
                    text = span["text"].strip()
                    if text:
                        font_key = (round(span["size"], 2), span["font"])
                        font_data[font_key].append({
                            "text": text,
                            "size": round(span["size"], 2),
                            "font": span["font"],
                            "page": p_idx + 1,
                            "bbox": span.get("bbox", None),
                            "position": (round(span["origin"][0], 1), round(span["origin"][1], 1))
                        })

    # Sort by font size or font name
    sorted_fonts = sorted(font_data.items(), key=lambda x: -x[0][0] if sort_by == "size" else x[0][1])

    # Build LLM-friendly string
    lines = []
    for (size, font), entries in sorted_fonts:
        line_of_text = f"Font Group: size={size}, font='{font}' (occurrences={len(entries)})"
        lines.append(line_of_text)
        if verbose > 0:
            print("\n"+line_of_text)
        for entry in entries[:examples_included]:  # Show top 5 examples per group
            line_of_text = f"  [Page {entry['page']}] (x={entry['position'][0]}, y={entry['position'][1]}) → {entry['text']}"
            lines.append(line_of_text)
            if verbose > 0:
                print(f"    [Page {entry['page']}] (x={entry['position'][0]}, y={entry['position'][1]}) → {entry['text']}")
        lines.append("")  # blank line between font groups

    return "\n".join(lines)

explore_pdf_fonts_llm_ready(file_path, sort_by="size", examples_included=3, verbose = 1)



Font Group: size=28.0, font='BoschSans-Regular' (occurrences=1)
    [Page 1] (x=31.5, y=279.2) → Washing machine

Font Group: size=20.0, font='BoschSans-Bold' (occurrences=4)
    [Page 52] (x=112.0, y=103.0) → Thank you for buying a
    [Page 52] (x=112.0, y=123.7) → Bosch Home Appliance!
    [Page 52] (x=112.0, y=327.6) → Looking for help?

Font Group: size=15.37, font='ArialUnicodeMS' (occurrences=1)
    [Page 19] (x=293.7, y=426.2) → 4

Font Group: size=14.51, font='ArialUnicodeMS' (occurrences=13)
    [Page 18] (x=38.6, y=341.7) → 1
    [Page 18] (x=38.6, y=277.9) → 2
    [Page 18] (x=136.7, y=161.8) → 4

Font Group: size=14.45, font='ArialUnicodeMS' (occurrences=9)
    [Page 19] (x=98.3, y=183.6) → 2
    [Page 19] (x=185.1, y=107.5) → 3
    [Page 19] (x=75.8, y=183.6) → 1

Font Group: size=14.0, font='NimbusSanL-Bol' (occurrences=31)
    [Page 2] (x=28.3, y=56.1) → Further information and explanations are
    [Page 2] (x=28.3, y=71.1) → available online:
    [Page 2] (x=28.3, y=1

'Font Group: size=28.0, font=\'BoschSans-Regular\' (occurrences=1)\n  [Page 1] (x=31.5, y=279.2) → Washing machine\n\nFont Group: size=20.0, font=\'BoschSans-Bold\' (occurrences=4)\n  [Page 52] (x=112.0, y=103.0) → Thank you for buying a\n  [Page 52] (x=112.0, y=123.7) → Bosch Home Appliance!\n  [Page 52] (x=112.0, y=327.6) → Looking for help?\n\nFont Group: size=15.37, font=\'ArialUnicodeMS\' (occurrences=1)\n  [Page 19] (x=293.7, y=426.2) → 4\n\nFont Group: size=14.51, font=\'ArialUnicodeMS\' (occurrences=13)\n  [Page 18] (x=38.6, y=341.7) → 1\n  [Page 18] (x=38.6, y=277.9) → 2\n  [Page 18] (x=136.7, y=161.8) → 4\n\nFont Group: size=14.45, font=\'ArialUnicodeMS\' (occurrences=9)\n  [Page 19] (x=98.3, y=183.6) → 2\n  [Page 19] (x=185.1, y=107.5) → 3\n  [Page 19] (x=75.8, y=183.6) → 1\n\nFont Group: size=14.0, font=\'NimbusSanL-Bol\' (occurrences=31)\n  [Page 2] (x=28.3, y=56.1) → Further information and explanations are\n  [Page 2] (x=28.3, y=71.1) → available online:\n  [Page 2] (x=2

In [ ]:
## Extracting section headers from the PDF files

def extract_text_chunks(file_path, chunk_size=512, chunk_overlap=200, embedding_vecor_size=1024):
    with pdfplumber.open(file_path) as pdf:
        full_text = "\n".join([page.extract_text() or "" for page in pdf.pages])

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = splitter.split_text(full_text)
    return [{'SECTION_ID': 0 ,"MANUAL_ID": 0, "CHUNK_TEXT": c, "CHUNK_ORDER": i} for i, c in enumerate(chunks)]

# extract_text_chunks(file_path, 512, 128)[:5]  # Show first 5 chunks

In [87]:
small_chunks_df.head()

,CHUNK_ID,SECTION_ID,MANUAL_ID,CHUNK_TEXT,CHUNK_ORDER
0,NaN,0,0,(cid:77)(cid:105)(cid:100)(cid:101)(cid:97)(ci...,0
1,NaN,0,0,(cid:49)(cid:32)(cid:83)(cid:65)(cid:70)(cid:6...,1
2,NaN,0,0,(cid:50)(cid:32)(cid:73)(cid:78)(cid:84)(cid:8...,2
3,NaN,0,0,(cid:32)(cid:32)(cid:32)(cid:32)(cid:50)(cid:4...,3
4,NaN,0,0,(cid:32)(cid:32)(cid:32)(cid:32)(cid:50)(cid:4...,4


In [ ]:
small_chunks_df = pd.DataFrame(columns=['CHUNK_ID','SECTION_ID', 'MANUAL_ID', 'CHUNK_TEXT', 'CHUNK_ORDER'])

# # fill the pandas dataframe with the metadata, and upload to snowflake
for filename in tqdm(os.listdir(pdf_files_path), total=len(os.listdir(pdf_files_path))):
    if filename.endswith(".pdf"):
        file_path = os.path.join(pdf_files_path, filename)
        chunks = extract_text_chunks(file_path = file_path, chunk_size=1024, chunk_overlap=128, embedding_vecor_size=1024)
        small_chunks_df = pd.concat([small_chunks_df, pd.DataFrame(chunks)], ignore_index=True)
    break # Just while testing

# Upload the small chunks to Snowflake
write_pandas(df = small_chunks_df, conn = conn, 
                database = "WASHING_MACHINE_MANUALS",
                schema="PUBLIC",
                table_name= "CHUNKS_SMALL",  
                auto_create_table=False, 
                overwrite=False)

print("Number of rows: ", len(small_chunks_df))
small_chunks_df.head()

  0%|          | 0/14 [00:05<?, ?it/s]


ProgrammingError: 001008 (22023): SQL compilation error:
invalid value [?] for parameter 'ON_ERROR'

In [86]:
small_chunks_df.head(40)

,CHUNK_ID,SECTION_ID,MANUAL_ID,CHUNK_TEXT,CHUNK_ORDER
0,NaN,0,0,(cid:77)(cid:105)(cid:100)(cid:101)(cid:97)(ci...,0
1,NaN,0,0,(cid:49)(cid:32)(cid:83)(cid:65)(cid:70)(cid:6...,1
2,NaN,0,0,(cid:50)(cid:32)(cid:73)(cid:78)(cid:84)(cid:8...,2
3,NaN,0,0,(cid:32)(cid:32)(cid:32)(cid:32)(cid:50)(cid:4...,3
4,NaN,0,0,(cid:32)(cid:32)(cid:32)(cid:32)(cid:50)(cid:4...,4
5,NaN,0,0,(cid:32)(cid:32)(cid:32)(cid:32)(cid:50)(cid:4...,5
6,NaN,0,0,(cid:32)(cid:32)(cid:32)(cid:32)(cid:50)(cid:4...,6
7,NaN,0,0,(cid:32)(cid:32)(cid:32)(cid:32)(cid:50)(cid:4...,7
8,NaN,0,0,(cid:32)(cid:32)(cid:32)(cid:32)(cid:50)(cid:4...,8
9,NaN,0,0,(cid:32)(cid:32)(cid:32)(cid:32)(cid:50)(cid:4...,9


In [ ]:
chunks_large_df = pd.DataFrame(columns=['SECTION_ID', 'MANUAL_ID', 'CHUNK_TEXT', 'CHUNKER_ORDER', 'EMBEDDING VECTOR'])

cursor.execute("""
UPDATE chunks_large
SET embedding = SNOWFLAKE.CORTEX.EMBED_TEXT('snowflake-arctic-embed-m', chunk_text)
WHERE VECTOR_NORM(embedding) = 0
;
""")

In [13]:
def render_pdf_to_images(pdf_path, zoom=2.0):
    doc = fitz.open(pdf_path)
    images = []
    for i, page in enumerate(doc):
        mat = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=mat)
        img_data = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        images.append({
            "page_number": i + 1,
            "image": img_data
        })
    return images


def detect_image_regions(page_image):
    image = np.array(page_image)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    regions = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 50 and h > 50:  # Skip tiny blocks (Maybe reconsider)
            regions.append([x, y, x + w, y + h])
    return regions


def crop_regions_from_image(page_image, regions, output_dir, page_num):
    os.makedirs(output_dir, exist_ok=True)
    saved_images = []

    for i, coords in enumerate(regions):
        x1, y1, x2, y2 = map(int, coords)
        cropped = page_image.crop((x1, y1, x2, y2))
        save_path = os.path.join(output_dir, f"page_{page_num}_img_{i+1}.png")
        cropped.save(save_path)
        saved_images.append({
            "page": page_num,
            "image_path": save_path,
            "coords": (x1, y1, x2, y2)
        })
    return saved_images


# This is the main function to extract images from the PDF
def extract_images_from_pdf(pdf_path:str, output_dir: str, verbose:int =0):
    rendered_pages = render_pdf_to_images(pdf_path)
    all_extracted = []

    for page_idx,page in enumerate(rendered_pages):
        page_num = page["page_number"]
        image = page["image"]
        if verbose > 0:
            print(f"Processing page {page_num}...")

        regions = detect_image_regions(image)
        if verbose > 0:
            print(f"Found {len(regions)} image regions on page {page_num}")

        if not regions:
            if verbose > 0:
                print(f"No image regions found on page {page_num}")
            continue
        
        # Creates an image directory for each PDF file
        image_output_dir = os.path.join(output_dir, pdf_path.split("/")[-1].replace(".pdf", ""))
        os.makedirs(image_output_dir, exist_ok=True)

        extracted = crop_regions_from_image(
            image, regions, output_dir=image_output_dir, page_num=page_num
        )
        all_extracted.extend(extracted)
    return all_extracted

extract_images_from_pdf(file_path, output_dir="Washer_Images", verbose = 1)

Processing page 1...
Found 1 image regions on page 1
Processing page 2...
Found 1 image regions on page 2
Processing page 3...
Found 0 image regions on page 3
No image regions found on page 3
Processing page 4...
Found 1 image regions on page 4
Processing page 5...
Found 0 image regions on page 5
No image regions found on page 5
Processing page 6...
Found 0 image regions on page 6
No image regions found on page 6
Processing page 7...
Found 0 image regions on page 7
No image regions found on page 7
Processing page 8...
Found 0 image regions on page 8
No image regions found on page 8
Processing page 9...
Found 0 image regions on page 9
No image regions found on page 9
Processing page 10...
Found 0 image regions on page 10
No image regions found on page 10
Processing page 11...
Found 0 image regions on page 11
No image regions found on page 11
Processing page 12...
Found 0 image regions on page 12
No image regions found on page 12
Processing page 13...
Found 7 image regions on page 13
Pro

[{'page': 1,
  'image_path': 'Washer_Images\\Washer_Manuals\\WGG254Z0GB\\page_1_img_1.png',
  'coords': (0, 0, 839, 1191)},
 {'page': 2,
  'image_path': 'Washer_Images\\Washer_Manuals\\WGG254Z0GB\\page_2_img_1.png',
  'coords': (656, 87, 778, 209)},
 {'page': 4,
  'image_path': 'Washer_Images\\Washer_Manuals\\WGG254Z0GB\\page_4_img_1.png',
  'coords': (56, 232, 142, 289)},
 {'page': 13,
  'image_path': 'Washer_Images\\Washer_Manuals\\WGG254Z0GB\\page_13_img_1.png',
  'coords': (144, 853, 224, 969)},
 {'page': 13,
  'image_path': 'Washer_Images\\Washer_Manuals\\WGG254Z0GB\\page_13_img_2.png',
  'coords': (62, 853, 142, 969)},
 {'page': 13,
  'image_path': 'Washer_Images\\Washer_Manuals\\WGG254Z0GB\\page_13_img_3.png',
  'coords': (464, 686, 788, 920)},
 {'page': 13,
  'image_path': 'Washer_Images\\Washer_Manuals\\WGG254Z0GB\\page_13_img_4.png',
  'coords': (62, 657, 224, 773)},
 {'page': 13,
  'image_path': 'Washer_Images\\Washer_Manuals\\WGG254Z0GB\\page_13_img_5.png',
  'coords': (62,